# Creating Tokens from Text

In [20]:
with open("01_main-chapter-code/the-verdict.txt") as f:
    raw_text = f.read()
print("Total number of characters in the text file: ", len(raw_text))
print(raw_text[:99])

Total number of characters in the text file:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [21]:
import re
result = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
result = [item for item in result if item.strip()] # Remove space strings
print(result[:30])
print(len(result))

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']
4690


# Converting Tokens into Token IDs

## Creating Vocabulary

In [37]:
all_words = sorted(set(result))  # Get unique words and sort them
vocab_size = len(all_words)
print("Total number of unique words in the text file: ", vocab_size)

Total number of unique words in the text file:  1130


In [38]:
vocab = {token: integer for integer, token in enumerate(all_words)}  # Create a dictionary of tokens and their IDs
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)
    if i > 10:
        break

('yet', 1125)
('you', 1126)
('younger', 1127)
('your', 1128)
('yourself', 1129)


## Implementing a Simple Text Tokenizer

In [13]:
class SimpleTokenizerV1:
    def __init__(self, vocabulary): # We must have a vocabulary to initialize the tokenizer
        self.str_to_int = vocabulary  # Dictionary of tokens and their IDs {"token1": 0, ...}
        self.int_to_str = {v: k for k, v in vocabulary.items()}  # Dictionary of IDs and their tokens {0: "token1", ...}
        
    def encode(self, text):
        # Convert text to a list of token IDs
        preprocessed_text = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed_text = [item.strip() for item in preprocessed_text if item.strip()]
        ids = [self.str_to_int[token] for token in preprocessed_text]
        return ids  # return a list of token IDs
    
    def decode(self, ids):
        # Convert a list of token IDs to text with spaces
        text = " ".join([self.int_to_str[id] for id in ids])
        
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)  # Remove spaces before punctuation
        return text

In [19]:
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)
print(tokenizer.decode(ids))

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]
" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


## SimpleTokenizerV1 doesn't handle unknown tokens. Create a new tokenizer SimpleTokenizerV2 that handles unknown tokens.

In [40]:
all_words.extend(["<|endoftext|>", "<|unk|>"])  # Add special tokens for end of text and unknown words
vocab = {token:integer for integer,token in enumerate(all_words)}  # Create a new vocabulary

for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1132)
('<|unk|>', 1133)


In [41]:
class SimpleTokenizerV2:
    def __init__(self, vocabulary): # We must have a vocabulary to initialize the tokenizer
        self.str_to_int = vocabulary  # Dictionary of tokens and their IDs {"token1": 0, ...}
        self.int_to_str = {v: k for k, v in vocabulary.items()}  # Dictionary of IDs and their tokens {0: "token1", ...}
        
    def encode(self, text) -> list:
        preprocessed_tokens = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed_tokens = [item.strip() for item in preprocessed_tokens if item.strip()]  # remove empty space strings and empty strings
        preprocessed_tokens = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed_tokens]  # Replace unknown words from source text with <|unk|>
        
        ids = [self.str_to_int[token] for token in preprocessed_tokens]
        return ids
    
    def decode(self, ids) -> str:
        decoded_text = " ".join([self.int_to_str[i] for i in ids])
        decoded_text = re.sub(r'\s+([,.?!"()\'])', r'\1', decoded_text) # Remove spaces before punctuation
        return decoded_text
        

In [43]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [45]:
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))
print(tokenizer.decode(tokenizer.encode(text)))

[1133, 5, 355, 1126, 628, 975, 10, 1132, 55, 988, 956, 984, 722, 988, 1133, 7]
<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


## A Tokenzier Which can Handle Any Unknown Words: Byte Pair Encoding (BPE)

In [46]:
# Use tiktoken library to implement BPE
import tiktoken
from importlib.metadata import version
print(version("tiktoken"))

0.5.2


In [55]:
tokenizer = tiktoken.get_encoding("gpt2")

In [59]:
text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace. Akwirw ier."
ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(ids)

text = tokenizer.decode(ids)
print(text)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13, 9084, 86, 343, 86, 220, 959, 13]
Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace. Akwirw ier.
